In [ ]:
import numpy as np

from tensorflow.keras.layers import (
    Dense,
    Dropout,
)
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import r2_score, mean_squared_error
import pickle

In [ ]:
X = np.load("../data/X_train_scaled.npy", allow_pickle=True)
y = np.load("../data/y_train_scaled.npy", allow_pickle=True)

In [ ]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (3104579, 26)
y shape: (3104579, 1)


In [ ]:
lr_reduce = ReduceLROnPlateau(
    min_delta=1e-5,
    patience=5,
    verbose=1
)

callbacks = [
    lr_reduce
]

In [ ]:
def create_model(n_neurons, input_shape=(X.shape[1], )):
    model = Sequential()
    model.add(Dense(n_neurons, activation="relu", input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(Dense(n_neurons, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation="linear"))
    model.compile(optimizer="adam", loss="mse")
    return model

In [ ]:
regressor = KerasRegressor(
    model=create_model,
    n_neurons=32,
    batch_size=128,
    epochs=100,
)

In [ ]:
param_grid = {
    "n_neurons": [32, 64, 128],
}

In [ ]:
grid_search = GridSearchCV(
    regressor,
    param_grid,
    scoring="neg_mean_squared_error",
    cv=10,
    verbose=10
)

In [ ]:
np.random.seed(2907)
grid_search.fit(X, y)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
[CV 1/10; 1/3] START n_neurons=32...............................................
Epoch 1/100
21830/21830 [==============================] - 42s 2ms/step - loss: 0.3439
Epoch 2/100
21830/21830 [==============================] - 42s 2ms/step - loss: 0.3139
Epoch 3/100
21830/21830 [==============================] - 43s 2ms/step - loss: 0.3115
Epoch 4/100
21830/21830 [==============================] - 42s 2ms/step - loss: 0.3100
Epoch 5/100
21830/21830 [==============================] - 43s 2ms/step - loss: 0.3080
Epoch 6/100
21830/21830 [==============================] - 43s 2ms/step - loss: 0.3073
Epoch 7/100
21830/21830 [==============================] - 43s 2ms/step - loss: 0.3068
Epoch 8/100
21830/21830 [==============================] - 42s 2ms/step - loss: 0.3063: 0s - loss: 0.306
Epoch 9/100
21830/21830 [==============================] - 42s 2ms/step - loss: 0.3059
Epoch 10/100
21830/21830 [==============================]

In [ ]:
print("GridSearch results:")
grid_search.cv_results_

In [ ]:
with open("../grids/grid_search_neural_net_01.pkl", mode="wb") as f:
    pickle.dump(grid_search, f)

In [ ]:
best_regressor = grid_search.best_estimator_

In [ ]:
print("Best regressor", best_regressor)

In [ ]:
with open("../models/model_gradient_neural_net_01.pkl", mode="wb") as f:
    pickle.dump(best_regressor, f)

In [ ]:
y_hat = best_regressor.predict(X)

In [ ]:
r2 = r2_score(y, y_hat)
mse = mean_squared_error(y, y_hat)

print(f"R2 Score: {r2:.3f}")
print(f"MSE: {mse:.3f}")